In [ ]:
import pandas as pd

In [ ]:
from datasets import Dataset
from ragas.metrics import answer_correctness
from ragas import evaluate


data_samples = {
    'question': [
        'When was the first Super Bowl?',
        'Which team has won the most Super Bowls?'
    ],
    'answer': [
        'The first Super Bowl was held on January 15, 1967.',
        'The New England Patriots have won the most Super Bowls.'
    ],
    'ground_truth': [
        'The first Super Bowl took place on January 15, 1967.',
        'The Pittsburgh Steelers have won the most Super Bowls.'
    ]
}

dataset = Dataset.from_dict(data_samples)
results = evaluate(dataset, metrics=[answer_correctness])

print(results)
